# AGENTS and TOOLS

`The core idea of agents is to use a language model to choose a sequence of actions to take.
In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.`

` Understanding:Agent performs iterative action with the help of llm by "Reasoning the input" and arraiving at a output with the help of "TOOL" provided to the agent`

## Load the ENV. & necessary Dependencies (llm,agent(tool),prompt(hub))

In [43]:
# Env
from dotenv import load_dotenv

# LLM
from langchain_openai import ChatOpenAI

# Agent
from langchain.agents import AgentExecutor , create_react_agent

# Prompts
from langchain import hub
from langchain.prompts.prompt import PromptTemplate

# tools
from langchain_core.tools import Tool
from langchain_community.tools.tavily_search import TavilySearchResults

# loading the .env variables
load_dotenv()

True

## Initialize the LLM to be used

In [31]:
 llm = ChatOpenAI(temperature=0,model='gpt-3.5-turbo')

## Template with output indicator

In [32]:
template = """given the full name {name_of_person} I want you to get me a link to theirr linkedin profile page.
            your answer should contain only a URL"""

In [33]:
template

'given the full name {name_of_person} I want you to get me a link to theirr linkedin profile page.\n            your answer should contain only a URL'

### Initialize Prompt Template for LLM

In [38]:
prompt_template = PromptTemplate(template=template,input_variables=['name_of_person'])
print(prompt_template)
dict(prompt_template)

input_variables=['name_of_person'] template='given the full name {name_of_person} I want you to get me a link to theirr linkedin profile page.\n            your answer should contain only a URL'


{'name': None,
 'input_variables': ['name_of_person'],
 'input_types': {},
 'output_parser': None,
 'partial_variables': {},
 'metadata': None,
 'tags': None,
 'template': 'given the full name {name_of_person} I want you to get me a link to theirr linkedin profile page.\n            your answer should contain only a URL',
 'template_format': 'f-string',
 'validate_template': False}

## Tool Initialization

In [44]:
def get_profile_url_tavily(name:str):
    """Searches for linkedin or Twitter profile page."""
    search = TavilySearchResults()
    response = search.invoke(f'{name}')
    return response[0]['url']

### List all the tools

In [45]:
tools_for_agent = [
    Tool(
        name='Crawl Google to get Linkedin profile page',
        func=get_profile_url_tavily,
        description="useful when you need to get the linkedin page URL" 
    )
]

## Create ReAct Prompt using LangChain HUB (Place to find prompt templates)

In [34]:
react_prompt = hub.pull('hwchase17/react')  #user/templateName

In [35]:
dict(react_prompt)  # the Input variables are filled by the langchain

{'name': None,
 'input_variables': ['agent_scratchpad', 'input', 'tool_names', 'tools'],
 'input_types': {},
 'output_parser': None,
 'partial_variables': {},
 'metadata': {'lc_hub_owner': 'hwchase17',
  'lc_hub_repo': 'react',
  'lc_hub_commit_hash': 'd15fe3c426f1c4b3f37c9198853e4a86e20c425ca7f4752ec0c9b0e97ca7ea4d'},
 'tags': None,
 'template': 'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}',
 'template_format': 'f-string',
 'validate_template

## Initializing the Agent

In [46]:
agent = create_react_agent(llm=llm,tools=tools_for_agent,prompt=react_prompt)

In [59]:
agent_executor = AgentExecutor(agent=agent,tools=tools_for_agent,verbose=True)

## Generating the Value from the PromptTemplate

In [60]:
prompt_template.format_prompt(name_of_person="kiran")

StringPromptValue(text='given the full name kiran I want you to get me a link to theirr linkedin profile page.\n            your answer should contain only a URL')

## Invoking the Agent -- AgentExecutor(Runtime for the agents)

In [61]:
result = agent_executor.invoke(input={
    "input":prompt_template.format_prompt(name_of_person="somesh yogaraj") #guess: "input" must be template from the hub
})
dict(result)



> Entering new AgentExecutor chain...
I should use the tool to crawl Google to get the Linkedin profile page.
Action: Crawl Google to get Linkedin profile page
Action Input: somesh yogaraj Linkedinhttps://www.linkedin.com/posts/someshyogaraj_im-happy-to-share-that-ive-obtained-a-new-activity-7103719312024076288-ZBYgI now know the final answer
Final Answer: https://www.linkedin.com/posts/someshyogaraj_im-happy-to-share-that-ive-obtained-a-new-activity-7103719312024076288-ZBYg

> Finished chain.


{'input': StringPromptValue(text='given the full name somesh yogaraj I want you to get me a link to theirr linkedin profile page.\n            your answer should contain only a URL'),
 'output': 'https://www.linkedin.com/posts/someshyogaraj_im-happy-to-share-that-ive-obtained-a-new-activity-7103719312024076288-ZBYg'}

## Ouput

In [54]:
linkedIn_profile_url = result['output']
print(linkedIn_profile_url)

https://www.linkedin.com/pub/dir/Somesh/Yogaraj
